In [2]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import gb_lag as lag
from sales_data import SalesData

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import LSTM

In [48]:
sd = SalesData()

sd.set_sales_data()
sd.merge_shop_data_to_sales()
sd.merge_item_data_to_sales()

Formatting and aggregating sales data...
Aggregating for monthly sales...


In [49]:
sd.daily_sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,year,month,year_month,sales_day,...,тц,трц,мега,тк,трк,молл,центральный,item_name,item_category_id,item_category_name
0,2013-01-02,0,59,22154,999.0,1.0,2013,01,201301,999.0,...,1,0,0,0,0,0,0,ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray
1,2013-01-02,0,25,22154,999.0,1.0,2013,01,201301,999.0,...,0,0,0,0,1,0,0,ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray
2,2013-01-03,0,25,22154,999.0,1.0,2013,01,201301,999.0,...,0,0,0,0,1,0,0,ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray
3,2013-01-20,0,25,22154,999.0,1.0,2013,01,201301,999.0,...,0,0,0,0,1,0,0,ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray
4,2013-01-23,0,25,22154,999.0,1.0,2013,01,201301,999.0,...,0,0,0,0,1,0,0,ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray


In [51]:
sd.daily_sales.shape

(2935835, 21)

In [52]:
sd.monthly_sales.head()

,date_block_num,year,month,year_month,shop_id,item_id,item_cnt_month,avg_price,sales_month,loc_name,тц,трц,мега,тк,трк,молл,центральный,item_name,item_category_id,item_category_name
0,0,2013,01,201301,0,1000,5.0,58.0,290.0,ЯКУТСК,0,0,0,0,0,0,0,"3D Action Puzzle ""Зомби"" Уборщик",67,Подарки - Развитие
1,1,2013,02,201302,0,1000,4.0,58.0,232.0,ЯКУТСК,0,0,0,0,0,0,0,"3D Action Puzzle ""Зомби"" Уборщик",67,Подарки - Развитие
2,0,2013,01,201301,1,1000,3.0,58.0,174.0,ЯКУТСК,1,0,0,0,0,0,1,"3D Action Puzzle ""Зомби"" Уборщик",67,Подарки - Развитие
3,1,2013,02,201302,1,1000,5.0,58.0,290.0,ЯКУТСК,1,0,0,0,0,0,1,"3D Action Puzzle ""Зомби"" Уборщик",67,Подарки - Развитие
4,5,2013,06,201306,10,1000,1.0,58.0,58.0,ЖУКОВСКИЙ,0,0,0,0,0,0,0,"3D Action Puzzle ""Зомби"" Уборщик",67,Подарки - Развитие


In [53]:
daily = sd.add_missing_dates(sd.daily_sales)

In [54]:
daily.shape

(2935835, 21)

In [46]:
sd.items.head()

,item_name,item_id,item_category_id,item_category_name
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40,Кино - DVD
1,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40,Кино - DVD
2,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40,Кино - DVD
3,***КОРОБКА (СТЕКЛО) D,4,40,Кино - DVD
4,***НОВЫЕ АМЕРИКАНСКИЕ ГРАФФИТИ (UNI) ...,5,40,Кино - DVD


In [44]:
sd._sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,year,month,year_month,sales_day,loc_name,тц,трц,мега,тк,трк,молл,центральный
0,2013-01-02,0,59,22154,999.0,1.0,2013,01,201301,999.0,ЯРОСЛАВЛЬ,1,0,0,0,0,0,0
1,2013-01-10,0,59,22151,399.0,1.0,2013,01,201301,399.0,ЯРОСЛАВЛЬ,1,0,0,0,0,0,0
2,2013-01-04,0,59,5603,699.0,1.0,2013,01,201301,699.0,ЯРОСЛАВЛЬ,1,0,0,0,0,0,0
3,2013-01-19,0,59,5587,199.0,2.0,2013,01,201301,398.0,ЯРОСЛАВЛЬ,1,0,0,0,0,0,0
4,2013-01-31,0,59,5613,5571.0,1.0,2013,01,201301,5571.0,ЯРОСЛАВЛЬ,1,0,0,0,0,0,0


In [47]:
sd.daily_sales.shape

(2935835, 18)

In [42]:
sd.shops.head()

,shop_id,loc_name,тц,трц,мега,тк,трк,молл,центральный
0,0,ЯКУТСК,0,0,0,0,0,0,0
1,1,ЯКУТСК,1,0,0,0,0,0,1
2,2,АДЫГЕЯ,1,0,1,0,0,0,0
3,3,БАЛАШИХА,0,0,0,0,1,0,0
4,4,ВОЛЖСКИЙ,1,0,0,0,0,1,0


In [ ]:
data = lag.feature_matrix(daily_threshold=500)

In [ ]:
data.head()

In [ ]:
train_df, test_df = lag.stack_and_train_test_split(data)

In [ ]:
test_df.head()

In [ ]:
X = train_df.iloc[:, :-1].copy()
y = train_df.iloc[:, -1:].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.20, random_state=123)

In [ ]:
# scale the data
sc_X = MinMaxScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)
sc_y = MinMaxScaler()
y_train = sc_y.fit_transform(y_train)

In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [ ]:
# build LSTM
model = Sequential()
model.add(LSTM(units=1, 
               batch_input_shape=(1, X_train.shape[1], X_train.shape[2]),
#                return_sequences=True,
#                return_state=True,
               dropout=0.1,
               stateful=True))
model.add(Dense(1))
model.add(Dense(1))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
model.fit(X_train, y_train, 
          epochs=30, 
          batch_size=1, 
          verbose=1, 
          shuffle=False)

In [ ]:
y_pred = model.predict(X_test, batch_size=1)

In [ ]:
y_pred_unscaled = sc_y.inverse_transform(y_pred)
print(f'RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_unscaled))}')

# Train on Entire Set

In [ ]:
# scale
sc_X = MinMaxScaler()
X = sc_X.fit_transform(X)
# X_test = sc_X.transform(X_test)
sc_y = MinMaxScaler()
y = sc_y.fit_transform(y)

X_r = np.reshape(X, (X.shape[0], 1, X.shape[1]))

In [ ]:
model.fit(X_r, y, 
          epochs=30, 
          batch_size=1, 
          verbose=1, 
          shuffle=False)

# Test Set
Merge test set to 2015 data. Then get predictions.

In [ ]:
test_set = pd.read_csv('data/test.csv')

test_set['shop_item'] = \
    test_set['shop_id'].astype(str) + '_' + test_set['item_id'].astype(str)

test_df = test_df.reset_index()

In [ ]:
test_df.head()

In [ ]:
test_input = pd.merge(test_set.loc[:, 'shop_item'], 
                    test_df, 
                    on='shop_item', how='left')

test_input.fillna(0, inplace=True)

In [ ]:
test_input.head()

In [ ]:
X_test = test_input.iloc[:, 1:]
X_test = sc_X.transform(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [ ]:
y_pred = model.predict(X_test, batch_size=1)
y_pred = sc_y.inverse_transform(y_pred)

In [ ]:
test_set['item_cnt_month'] = y_pred
test_set.head()

In [ ]:
output = test_set.loc[:, ['ID', 'item_cnt_month']].copy()
output.set_index(keys='ID', inplace=True)
output = output.clip(0, 20)

output.to_csv('output/lstm_all_data_v2.csv', index=True)